In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [31]:
torch.manual_seed(42)

In [32]:
df = pd.read_csv('fmnist_small.csv')

In [33]:
x = df.iloc[:,1:].values
y = df.iloc[:, 0].values

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [35]:
x_train = x_train/255.0
y_train = y_train/255.0

In [36]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    

In [37]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

In [38]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [39]:
class CustomCNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7,128),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(64,10),
            nn.ReLU(),
        )
    
    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    

In [40]:
lr = 0.01
epochs = 100

In [41]:
model = CustomCNN(1)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = lr, weight_decay=1e-4)

In [42]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for features, labels in train_loader:
        y_pred = model(features)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_epoch_loss += loss.item()
    
    avg_loss = total_epoch_loss/len(train_loader)
    print(f"Epoch: {epoch + 1} || Loss: {avg_loss}")
    

Epoch: 1 || Loss: 0.10918572115401427
Epoch: 2 || Loss: 0.002327729512471706
Epoch: 3 || Loss: 0.0014550543039755818
Epoch: 4 || Loss: 0.000920586263940398
Epoch: 5 || Loss: 0.0006076243669879963
Epoch: 6 || Loss: 0.00040417961663479217
Epoch: 7 || Loss: 0.0005729707181975148
Epoch: 8 || Loss: 0.0004432336704849149
Epoch: 9 || Loss: 0.00038895056444744115
Epoch: 10 || Loss: 0.00028442557599494953
Epoch: 11 || Loss: 0.0003189128367921512
Epoch: 12 || Loss: 0.00018029793545565554
Epoch: 13 || Loss: 0.00020952269874518
Epoch: 14 || Loss: 0.0002570355413422476
Epoch: 15 || Loss: 0.00021233867641285543
Epoch: 16 || Loss: 0.00016385486732133358
Epoch: 17 || Loss: 0.00019289721019125257
Epoch: 18 || Loss: 0.00020330523151187662
Epoch: 19 || Loss: 0.00013346350029488956
Epoch: 20 || Loss: 0.00017874048741229367
Epoch: 21 || Loss: 0.00011595392949175221
Epoch: 22 || Loss: 8.628651964878979e-05
Epoch: 23 || Loss: 6.486869913298202e-05
Epoch: 24 || Loss: 0.00021129763346910597
Epoch: 25 || Loss: 

In [43]:
model.eval()

CustomCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
    (8): ReLU()
  )
)

In [50]:
total = 0
correct = 0
with torch.no_grad():
    for features, labels in test_loader:
        y_pred = model(features)
        _, pred = torch.max(y_pred, 1)
        total+=labels.shape[0]
        correct += (pred==labels).sum().item()

print(total, correct)
print(f"Acc: {correct/total}")

1200 146
Acc: 0.12166666666666667


In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for features, labels in train_loader:
        y_pred = model(features)
        _, pred = torch.max(y_pred, 1)
        total+=labels.shape[0]
        
        correct += (pred==labels).sum().item()
    
print(f"Acc: {correct/total}")

Acc: 1.0
